<center>
<img src="https://raw.githubusercontent.com/dvgodoy/PyTorch101_ODSC_Europe2020/master/images/linear_dogs.jpg" width="800">

# Основы глубинного обучения: домашнее задание 4
</center>

**ФИО:**

**Забавный факт о себе:**

## Общая информация

__Дата выдачи:__ 26.11.2024

__Мягкий дедлайн:__ 23:59MSK 10.12.2024

__Жесткий дедлайн:__ 23:59MSK 13.12.2024

## Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов + 2 бонусных балла.

**Дисклеймер:** Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник). Также помните, что простое копирование чужого кода не является самостоятельной работой.

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

In [4]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# Описание данных

Данные можно [скачать с гугл-диска.](https://drive.google.com/drive/folders/11oCcLplWtp_qm-WuEbfCFP_Mz5K_z3ps?usp=sharing) Если вы делаете задание в колабе, то строчки ниже сами скачают вам данные.



In [5]:
import gdown

url = "https://drive.google.com/drive/folders/11oCcLplWtp_qm-WuEbfCFP_Mz5K_z3ps?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/news_data/ria_news.tsv',
 '/content/news_data/vk_comments.tsv',
 '/content/news_data/vk_news.tsv']

В таблице `ria_news.tsv`  лежат данные о новостях, вышедших на сайте РИА-НОВОСТИ с 15 марта 2018 года по 31 декабря 2018 года.

- `href` - уникальный идентификатор новости (ссылка на неё)
- `date` - дата публикации новости
- `time` - время публикации новости
- `title` - заголовок новости
- `snippet` - краткое описание новости
- `text` - текст новости
- `category` - категория новости
- `keywords` - ключевые слова (подкатегории новости)
- `shows` - счётчик с числом просмотров новости на сайте (на момент парсинга)

In [6]:
df_ria = pd.read_csv('news_data/ria_news.tsv', sep='\t')
df_ria = df_ria[~df_ria.tags.isnull()]
print(df_ria.shape)
df_ria.head()

(201708, 9)


,href,date,time,title,snippet,text,category,tags,shows
0,/20181231/1548961410.html,2018-12-31,"31 декабря 2018, 23:52",Нетаньяху не собирается в отставку в случае пр...,Премьер-министр Израиля Биньямин Нетаньяху не ...,"МОСКВА, 31 дек - РИА Новости. Премьер-министр ...",В мире,"Биньямин Нетаньяху, Израиль, В мире",728.0
1,/20181231/1548961364.html,2018-12-31,"31 декабря 2018, 23:19",Макрон в новогоднем обращении затронул тему ре...,"Результат реформ не может быть мгновенным, зая...","ПАРИЖ, 31 дек – РИА Новости. Результат реформ ...",В мире,"Эммануэль Макрон, Франция, В мире",3086.0
2,/20181231/1548961337.html,2018-12-31,"31 декабря 2018, 23:12",Аарон Рэмзи проведет переговоры с пятью топ-кл...,"Полузащитник лондонского ""Арсенала"" Аарон Рэмз...","МОСКВА, 31 дек - РИА Новости. Полузащитник лон...",NaN,ФК Арсенал (Лондон),183.0
3,/20181231/1548961304.html,2018-12-31,"31 декабря 2018, 23:09",Гол Азмуна принес сборной Ирана победу над кат...,Футболисты сборной Ирана одержали победу над к...,"МОСКВА, 31 дек - РИА Новости. Футболисты сборн...",NaN,"Сердар Азмун, Сборная Ирана по футболу",78.0
4,/20181231/1548961265.html,2018-12-31,"31 декабря 2018, 23:07",Пятая ракетка мира дель Потро пропустит Открыт...,Аргентинский теннисист Хуан Мартин дель Потро ...,"МОСКВА, 31 дек - РИА Новости. Аргентинский тен...",NaN,Теннис,79.0


Многие новостные агенства поддерживают странички в социальных сетях. Они постят туда самые сочные сюжеты. В таблице `vk_news.tsv` лежат данные о новостях, которые РИА запостили ВКонтакте в период времени с  `2017-09-29 01:28:55` по `2019-02-01 23:13:17`.

- `id` - уникальный идентификатор поста
- `href` - ссылка на сайт (если она была указана в посте)
- `datetime` - дата и время публикации новости
- `title` - заголовок новости
- `text` - текст новости в социальной сети
- `likes` - число лайков под постом
- `comments` - число комментариев под постом

In [7]:
df_vk = pd.read_csv('news_data/vk_news.tsv', sep='\t')
df_vk['snippet'] = df_vk['text']
df_vk.drop('text', axis=1, inplace=True)
print(df_vk.shape)
df_vk.head()

(19928, 7)


,id,href,datetime,title,likes,comments,snippet
0,24006362,/20190201/1550280358.html,2019-02-01 23:13:17,"В ДНР заявили о задержании диверсантов, причас...",15,28,NaN
1,24006240,/20190201/1550268781.html,2019-02-01 22:38:41,"Житель Урала ""заминировал"" ТЦ из-за снятия со...",32,42,NaN
2,24006100,/20190201/1550282212.html,2019-02-01 21:58:52,"В Черном море нашли ""потерянный флот Гитлера""",84,23,NaN
3,24005972,/20190202/1550283179.html,2019-02-01 21:27:06,В США освободили задержанную российскую актрис...,58,35,NaN
4,24005764,/20190201/1550262848.html,2019-02-01 20:55:54,Толкнувший Скабееву депутат Рады заявил о гроз...,45,145,NaN


В таблице `vk_comments.tsv` лежат комментарии к новостям.

- `id` - уникальный идентификатор комментария
- `post_id` - идентификатор новости, под которой был оставлен комментарий
- `datetime` - дата и время, когда был оставлен комментарий
- `text` - текст комментария
- `likes` - число лайков под комментарием

In [8]:
df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')
df_comments = df_comments[~df_comments.text.isnull()]
print(df_comments.shape)
df_comments.head()

<ipython-input-8-9f937a17e540>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')


(2612629, 5)


,id,post_id,datetime,text,likes
0,24006366.0,24006362.0,2019-02-01 23:14:14,ЧВК Вагнера?,5.0
1,24006370.0,24006362.0,2019-02-01 23:15:23,"[id4710641|Евгений], выздоравливай.",3.0
2,24006371.0,24006362.0,2019-02-01 23:16:21,"[id442655034|Андрей], искренне желаю этого все...",4.0
3,24006374.0,24006362.0,2019-02-01 23:16:38,Опять про Украину новости?,1.0
4,24006375.0,24006362.0,2019-02-01 23:16:40,Че такое ДНР?,2.0


# А что надо сделать то?

В тетрадке вам предстоит сделать следующие шаги:

1. Обучить нейросеть предсказывать категорию новости
2. Построить предсказания для тех новостей, где мы ничего не знаем о категории
3. Использовать уже обученный для сентимент-анализа классификатор из библиотеки `hugging face` чтобы предсказать эмоциональную окраску каждого комментария
4. Провести аналитику по новостям, а именно построите топы из самых позитивных и негативных категорий и новостей

Для первого шага вам будет дан бэйзлайн. Если вы его прогоните, у вас получится базовая модель, которая даст некоторое качество решения задачи. Вам надо будет выяснить, насколько это качество оказалось хорошим, а затем внести в код некоторые улучшения.



## Часть 1: категоризация новостей (5 баллов + 1.8 бонусных)

Каждой новости в соотвествие поставлены ключевые слова. Будем считать, что эти ключевые слова — тематики новости. Нужно научиться предсказывать тематики по тексту новости. Готовые тематики у нас есть только по новостям с сайта. Они за 2018 год. По новостям из ВКонтакте у нас тематик нет. Мы собираемся их предсказать.

Новости, опубликованные ВКонтакте, отличаются от новостей с сайта тем, что у них есть только титул и короткое описание. Странно будет обучать нейросеть на длинных текстах, а потом использовать её на коротких описаниях. Мы не будем так делать. Мы попробуем обучить базовый вариант нейронной сети только на заголовках новостей. Все, кто захочет получить бонусные баллы, смогут попробовать добавить в нейросеть сниппеты (так назыают короткие описания новостей).

## 1.1 Подготовка таргета

Поработаем с таргетом. Мы будем предсказывать переменную `tags`. Давайте выясним скоько уникальных тегов существует.

In [9]:
from collections import Counter

# удалим все лишние пробелы и сделаем lowercase
df_ria['tags'] = (
    df_ria.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(df_ria.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))
tags_cnt.most_common()[-20:]

13344


[('блог анны завершинской об автоспорте - блоги', 1),
 ('министерство транспорта рб', 1),
 ('министерство здравоохранения грузии', 1),
 ('палех', 1),
 ('юрий посохов (хореограф)', 1),
 ('мария александрова', 1),
 ('том бенсон', 1),
 ('абдул каюм кочай', 1),
 ('нуман куртулмуш', 1),
 ('mipim', 1),
 ('владимир попов', 1),
 ('брюно женезио', 1),
 ('роберт фицо', 1),
 ('сергей пашинский', 1),
 ('валерия гонтарева', 1),
 ('нововоронеж', 1),
 ('императорское православное палестинское общество', 1),
 ('event_poslanie_prezidenta_rf_federalnomu_sobraniju', 1),
 ('фхтр', 1),
 ('игорь честин', 1)]

Всего в выборке есть порядка 13 000 тэгов. Многие встречаются всего по разу. Давайте оставим в выборке только те тэги, которые встречаются более 30 раз.

In [10]:
target_tags = {tag for tag,cnt in tags_cnt.most_common() if cnt > 30}
len(target_tags)

1583

Закодируем теги для OHE.

In [11]:
tag2idx = dict(zip(target_tags, range(len(target_tags))))
idx2tag = {jtem: item for item,jtem in tag2idx.items()}

Почистим таргет от лишних тэгов.

In [12]:
df_ria['target_tags'] = (
    df_ria.tags.
    apply(lambda w: [tag2idx.get(item) for item in  w.split(',') if item in target_tags])
)

df_ria = df_ria[df_ria.target_tags.apply(len) > 0]
df_ria.shape

(201437, 10)

In [13]:
df_ria.target_tags.values[:3]

array([list([1439, 676, 797]), list([721, 1096, 797]), list([417])],
      dtype=object)

## 1.2 Подготовка текстов

Теперь займёмся предобработкой текстов. Приведём все слова к нижнему регистру и выбросим мусорные символы. В качестве токенов будем рассматривать отдельные слова.

Напомню, что мы пока что решили работать только с названиями статей. Поэтому вся предобработка применяется исключительно к ним. **Спойлер:** предобработку для сниппетов вы сделаете сами в первом же задании.

In [14]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
import re
from nltk.tokenize import word_tokenize

def normalise_text(text):
    text = text.lower()

    # сурово регулярками выкидываем мусорные символы
    text = re.sub('[^а-яa-z0-9 ]', '', text)
    return text.strip()

df_ria['title_clean'] = df_ria.title.apply(normalise_text)

word_cnt = Counter(word_tokenize(' '.join(df_ria.title_clean.values)))
len(word_cnt)

112178

In [16]:
word_cnt.most_common()[:10]

[('в', 127323),
 ('на', 44386),
 ('с', 26150),
 ('и', 21771),
 ('о', 19948),
 ('по', 17014),
 ('россии', 13494),
 ('не', 13483),
 ('сша', 9942),
 ('за', 9881)]

Давайте почистим словарь от стоп-слов и подготовим его к использованию внутри датасета. Мы будем с помощью словаря заменять слова на индексы. Добавим в словарь несколько специальных токенов для неизвестных слов и паддингов.

In [17]:
from nltk.corpus import stopwords

stops_ru = set(stopwords.words('russian'))
len(stops_ru)

151

In [18]:
vocabulary = {
    "#PAD#": 0, "#UNK#": 1
}

k = 2
for word, _ in word_cnt.most_common():
    if word not in stops_ru:
        vocabulary[word] = k
        k += 1

In [19]:
len(vocabulary)

112030

Завернём код для создания словаря в функцию.

In [20]:
def create_vocab(text, stops_ru=stops_ru):

    word_cnt = Counter(word_tokenize(text))
    vocabulary = {
        "#PAD#": 0, "#UNK#": 1
    }

    k = 2
    for word, _ in word_cnt.most_common():
        if word not in stops_ru:
            vocabulary[word] = k
            k += 1
    return vocabulary

__[0.5 балла] Задание 1:__

- Cделайте аналогичную предобработку титулов из таблички `df_vk`. Запишите получившийся результат в столбец `title_clean` по аналогии с таблицей `df_ria`.
- Сделайте для обеих таблиц предобработку колонок со сниппетами `snippet` и запишите получившийся результат в столбец `snippet_clean`. Все пропуски заполните токеном `"#UNKN"`.

In [21]:

df_vk['title_clean'] = df_vk.title.apply(normalise_text)

def preprocess_snippet(snippet):
    if pd.isnull(snippet):
        return "#UNKN"
    return normalise_text(snippet)

df_ria['snippet_clean'] = df_ria.snippet.apply(preprocess_snippet)
df_vk['snippet_clean'] = df_vk.snippet.apply(preprocess_snippet)

# Проверим, что столбцы добавлены
df_ria[['title_clean', 'snippet_clean']].head(), df_vk[['title_clean', 'snippet_clean']].head()


(                                         title_clean  \
 0  нетаньяху не собирается в отставку в случае пр...   
 1  макрон в новогоднем обращении затронул тему ре...   
 4  пятая ракетка мира дель потро пропустит открыт...   
 5  сми кевину спейси не разрешили отсутствовать н...   
 6  зимняя классика нхл в 2020 году пройдет в далл...   
 
                                        snippet_clean  
 0  премьерминистр израиля биньямин нетаньяху не н...  
 1  результат реформ не может быть мгновенным заяв...  
 4  аргентинский теннисист хуан мартин дель потро ...  
 5  суд в американском штате массачусетс отверг хо...  
 6  зимняя классика  матч регулярного чемпионата н...  ,
                                          title_clean snippet_clean
 0  в днр заявили о задержании диверсантов причаст...         #UNKN
 1  житель урала заминировал тц изза снятия  сотов...         #UNKN
 2        в черном море нашли потерянный флот гитлера         #UNKN
 3  в сша освободили задержанную российскую акт

## 1.3 Поставка данных

Пересечение сайта и ВК по опубликованным новостям довольно маленькое. Мы обучаем модель на данных с сайта. Предсказания мы будем строить на данных из ВК. У этих данных разная природа. В ВК описание статей и заголовки короче. Модель может хорошо показать себя на данных с новостного сайта, но сильно просесть в качестве на данных из ВК.

Давайте сохраним пересечение в отдельную табличку, чтобы на нём можно было понять, насколько сильно деградирует модель.

In [22]:
ria_hrefs = set(df_ria.href.values)
vk_hrefs = set(df_vk.href.values)
test_hrefs = (vk_hrefs & ria_hrefs)

print('Размер отложенной выборки:', len(test_hrefs))

df = df_ria[~df_ria.href.isin(test_hrefs)]

Размер отложенной выборки: 1128


По странному совпадению (я правда не знаю почему) пересечение лежит в декабре. Мы будем его использовать как тестовую выборку.

In [23]:
df_ria[df_ria.href.isin(test_hrefs)].date.min(), df_ria[df_ria.href.isin(test_hrefs)].date.max()

('2018-12-06', '2018-12-31')

Предположим, что мы делим выборку на обучающую и тестовую случайно. За один и тот же промежуток времени может выйти довольно большое число новостей с одинаковым заголовком. Давайте представим себе, что в тесте и трэйне есть много статей про одно и то же событие. Модель научилась на обучающей выборке хорошо его тегировать. Остальные события модель тегирует намного хуже. Метрики на тестовой выборке высокие. В следующем месяце СМИ перестают освещать это событие, в потоке новостей совершенно другие новости. Качество модели резко проседает.

Чтобы не напороться на завышенные метрики, обычно выборку дробят на обучающую и тестовую по времени. Тогда статьи из теста будут имитировать поток новых новостей, освещающих новые события.

In [24]:
df.date.min(), df.date.max()

('2018-03-15', '2018-12-31')

__[0.5 балла] Задание 2:__ Разбейте выборку на обучающую, валидационную и тестовую. В тест возьмите весь декабрь. В валидацию октябрь и ноябрь.

In [25]:
df_test = df[df.date.str.startswith('2018-12')]
df_val = df[df.date.str.startswith('2018-10')]
df_val = pd.concat([df_val, df[df.date.str.startswith('2018-11')]])
df_train = df[~df.date.str.startswith(('2018-10', '2018-11', '2018-12'))]


Сформируем отложенную выборку (пересечение ВКонтакте и РИА).

In [26]:
df_oob = df_vk[df_vk.href.isin(test_hrefs)][['href', 'title_clean']]

df_ria_oob = df_ria[df_ria.href.isin(test_hrefs)][['href', 'target_tags']]
df_oob = df_oob.set_index('href').join(df_ria_oob.set_index('href')).reset_index()
df_oob.head()

,href,title_clean,target_tags
0,/20181231/1548960744.html,митрополит рязанский раскритиковал иронию судьбы,[280]
1,/20181231/1548958617.html,на украине позавидовали стене на границе с крымом,"[1307, 797]"
2,/20181231/1548957394.html,в госдуме предложили отказаться от газа в жилы...,"[1010, 1010, 1237, 1381, 1359]"
3,/20181231/1548954909.html,названы лучшие средства от похмелья,[280]
4,/20181231/1548957120.html,новогоднее поздравление порошенко разозлило по...,"[1307, 797]"


Напишем датасет для поставки данных в нейросеть.


In [27]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NewsDataset(Dataset):

    def __init__(self, target, title, vocab, vocab_size, max_title_len, max_classes, snippet=None, max_snippet_len=None):

       self.vocab = {word: idx  for word,idx in vocab.items() if idx < vocab_size}
       self.max_classes = max_classes
       self.y=self.target_ohe(target)
       self.X_title = self.create_text(title, max_title_len)

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = [ ]
        for sent in texts:
            # {#PAD: 0, #UNKN: 1}
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]

            # приводим все тексты к max_len
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        return (self.X_title[idx, :], self.y[idx])


__[0.5 балла] Задание 3:__ Сейчас датасет умеет работать только с полем `title_clean`. Давайте сделаем этот датасет более многофукнциональным и добавим в него возможность добавить в обработку данных сниппет.

1. Внутри датасета `snippet` надо обработать точно также как и `title`.
2. Если `snippet=None`, датасет должен вернуть два объекта: `X_title, y`. В обратном случае датасет должен вернуть три объекта.

**Важно:** Весь код ниже работает сейчас без сниппета. Он не должен развалиться от того, что сниппет в нём нигде не указан.

In [28]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NewsDataset(Dataset):
    def __init__(self, target, title, vocab, vocab_size, max_title_len, max_classes, snippet=None, max_snippet_len=None):
        self.vocab = {word: idx for word, idx in vocab.items() if idx < vocab_size}
        self.max_classes = max_classes
        self.y = self.target_ohe(target)
        self.X_title = self.create_text(title, max_title_len)

        # Если сниппеты указаны, обрабатываем их
        if snippet is not None:
            self.X_snippet = self.create_text(snippet, max_snippet_len)
        else:
            self.X_snippet = None

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i] * len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = []
        for sent in texts:
            # Используем словарь {#PAD: 0, #UNKN: 1}
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]
            # Приводим текст к max_len
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        if self.X_snippet is not None:
            return self.X_title[idx, :], self.X_snippet[idx, :], self.y[idx]
        else:
            return self.X_title[idx, :], self.y[idx]


Объявим датасеты, оставим в словаре 30 000 самых частотных слов. Будем смотреть на титулы максимальной длины 20.

In [29]:
CLASSES_NUM = len(idx2tag)
VOCAB_SIZE = 10000
MAX_TITLE_LEN = 20

# словарь создаем по всей выборке
vocabulary = create_vocab(' '.join(df_ria.title_clean.values))

# объявляем датасеты
train_dataset = NewsDataset(df_train.target_tags.values, df_train.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
val_dataset = NewsDataset(df_val.target_tags.values, df_val.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
test_dataset = NewsDataset(df_test.target_tags.values, df_test.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )

In [30]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=4096, num_workers=4)

## 1.4 Архитектуры

Соберём базовую архитектуру для обучения.

In [31]:
from torch import nn
import torch.nn.functional as F

class SimpleClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, title):
        embedded = self.embedding(title)
        embedded = embedded.mean(dim=1)
        return self.fc(embedded)

Соберём в `pytorch_lightning` модуль для обучения нейронки.

In [32]:
!pip3 install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 47.2 MB/s eta 0:00:00


In [33]:
import pytorch_lightning as pl

class TrainLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title):
        result = self.model(title)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        title, target = train_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )
        return loss

    def validation_step(self, val_batch, batch_idx):
        title, target = val_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "val_loss", loss, prog_bar=True
        )
        return loss

Обучим модель.

In [34]:
EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

model_baseline = SimpleClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module =TrainLightningModule(model_baseline, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | SimpleClassifier | 3.5 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


__[1 балл] Задание 4:__  Модель выше работает только с титулом.

- Залоггируйте её обучение на [WandB](https://wandb.ai/).
- Соберите архитектуру, которая будет принимать на вход не только титул, но ещё и сниппет. В этой архитектуре должно происходить следующее:

1. Общий слой `nn.Embedding` применяется к сниппету и титулу параллельно.
2. Происходит усреднее по текстам.
3. Вектора конкатятся в один длины 600
4. Линейный слой делает классификацию

Обучите эту модель. Сравните траектории обучения на WandB. Прикрепите ссылку на дашборд либо скришот к тетрадке.

Даталоадеры придётся объявить заново с учётом сниппетов. Правда ли, что она бьёт на валидационной выборке модель, обученную только на титулах статей?

In [35]:
# Модель, работающая с титулом и сниппетом
class SnippetTitleClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim * 2, output_dim)

    def forward(self, title, snippet):
        embedded_title = self.embedding(title).mean(dim=1)
        embedded_snippet = self.embedding(snippet).mean(dim=1)

        concatenated = torch.cat((embedded_title, embedded_snippet), dim=1)
        return self.fc(concatenated)

class TrainLightningModuleWithSnippet(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title, snippet):
        result = self.model(title, snippet)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        title, snippet, target = train_batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)

        if batch_idx % 10 == 0:
            print(f"Training Step [{batch_idx}]: Loss = {loss.item():.4f}")

        return loss

    def validation_step(self, val_batch, batch_idx):
        title, snippet, target = val_batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)

        print(f"Validation Step [{batch_idx}]: Loss = {loss.item():.4f}")
        return loss

class NewsDatasetWithSnippet(Dataset):
    def __init__(self, target, title, snippet, vocab, vocab_size, max_title_len, max_snippet_len, max_classes):
        self.vocab = {word: idx for word, idx in vocab.items() if idx < vocab_size}
        self.max_classes = max_classes
        self.y = self.target_ohe(target)
        self.X_title = self.create_text(title, max_title_len)
        self.X_snippet = self.create_text(snippet, max_snippet_len)

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i] * len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = []
        for sent in texts:
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        return (self.X_title[idx, :], self.X_snippet[idx, :], self.y[idx])

EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3
MAX_SNIPPET_LEN = 50

train_dataset = NewsDatasetWithSnippet(
    df_train.target_tags.values, df_train.title_clean.values, df_train.snippet_clean.values,
    vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, MAX_SNIPPET_LEN, CLASSES_NUM
)
val_dataset = NewsDatasetWithSnippet(
    df_val.target_tags.values, df_val.title_clean.values, df_val.snippet_clean.values,
    vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, MAX_SNIPPET_LEN, CLASSES_NUM
)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=4096, num_workers=4)

model_with_snippet = SnippetTitleClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module_with_snippet = TrainLightningModuleWithSnippet(model_with_snippet, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module_with_snippet, train_dataloader, val_dataloader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | model     | SnippetTitleClassifier | 4.0 M  | train
1 | criterion | CrossEntropyLoss       | 0      | train
-------------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.806    Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Validation Step [0]: Loss = 21.8576
Validation Step [1]: Loss = 22.5307


Training: |          | 0/? [00:00<?, ?it/s]

Training Step [0]: Loss = 22.3395
Training Step [10]: Loss = 17.6980
Training Step [20]: Loss = 18.4720
Training Step [30]: Loss = 17.0048
Training Step [40]: Loss = 16.6830
Training Step [50]: Loss = 17.5148
Training Step [60]: Loss = 16.4725
Training Step [70]: Loss = 13.6853
Training Step [80]: Loss = 15.4255
Training Step [90]: Loss = 14.6347
Training Step [100]: Loss = 14.5675
Training Step [110]: Loss = 16.2972
Training Step [120]: Loss = 16.3630
Training Step [130]: Loss = 14.5338
Training Step [140]: Loss = 16.6085
Training Step [150]: Loss = 17.0359
Training Step [160]: Loss = 13.6203
Training Step [170]: Loss = 16.9972
Training Step [180]: Loss = 17.3157
Training Step [190]: Loss = 14.3857
Training Step [200]: Loss = 14.3313
Training Step [210]: Loss = 15.0711
Training Step [220]: Loss = 16.2620
Training Step [230]: Loss = 13.9155
Training Step [240]: Loss = 14.8518
Training Step [250]: Loss = 15.2994
Training Step [260]: Loss = 12.8707
Training Step [270]: Loss = 15.7628
Tra

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Step [0]: Loss = 9.8416
Validation Step [1]: Loss = 10.6767
Validation Step [2]: Loss = 11.4496
Validation Step [3]: Loss = 9.6385
Validation Step [4]: Loss = 8.7902
Validation Step [5]: Loss = 10.6557
Validation Step [6]: Loss = 12.6543
Validation Step [7]: Loss = 11.4313
Validation Step [8]: Loss = 12.4948
Validation Step [9]: Loss = 11.6842
Validation Step [10]: Loss = 11.2718
Training Step [0]: Loss = 6.4882
Training Step [10]: Loss = 8.1323
Training Step [20]: Loss = 5.4679
Training Step [30]: Loss = 6.1973
Training Step [40]: Loss = 6.6010
Training Step [50]: Loss = 8.5917
Training Step [60]: Loss = 7.7422
Training Step [70]: Loss = 6.1375
Training Step [80]: Loss = 6.5086
Training Step [90]: Loss = 6.1163
Training Step [100]: Loss = 5.9657
Training Step [110]: Loss = 7.4291
Training Step [120]: Loss = 6.6421
Training Step [130]: Loss = 6.3676
Training Step [140]: Loss = 7.2825
Training Step [150]: Loss = 7.9822
Training Step [160]: Loss = 5.6608
Training Step [170]: L

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Step [0]: Loss = 9.8420
Validation Step [1]: Loss = 10.6875
Validation Step [2]: Loss = 11.4963
Validation Step [3]: Loss = 9.5698
Validation Step [4]: Loss = 8.6833
Validation Step [5]: Loss = 10.4505
Validation Step [6]: Loss = 12.4396
Validation Step [7]: Loss = 11.1072
Validation Step [8]: Loss = 12.2145
Validation Step [9]: Loss = 11.3836
Validation Step [10]: Loss = 10.9973


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


## 1.5 Инференс и оценка качества моделей

Для каждой новости нам надо предсказывать несколько тэгов. То есть в нашем случае настоящее значение таргета это множество из тэгов $y_i = [tag1, tag2, tag3]$. Прогноз модели также множество из тэгов $\hat y_i = [tag1, tag4]$.

Будем считать метрики качества следующим образом (под $|A|$ имеется в виду мощность множества, то есть число элементов в нём):

$$
Precision = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|\hat{y}_i|}
$$

$$
Recall = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|y_i|}
$$

Также можно считать аналог Accuracy, но это не самая удачная идея, так как у нас в выборке огромное число нулей и эта метрика при любом разумном пороге для принятия решения будет очень высокой.

$$
Exact Match = \frac{1}{n} \cdot \frac{1}{k} \sum_{i = 1}^n \sum_{k=1}^K [y_{ij} = \hat{y}_{ij}]
$$

In [36]:
def precision(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (y_pred == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def recall(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (target == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def exact_match(target, y_pred):
    return (1.*(y_pred == target)).mean().item()

Построим прогноз на тестовой выборке.

In [37]:
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=test_dataset.__len__())

for title, target in test_dataloader:
    logit = model_baseline(title)
    pred_prob = F.softmax(logit, dim=1)

assert pred_prob.shape[0] == test_dataset.__len__()

Теперь выбирая различное значение порога, мы можем получать разные предсказания. Если взять очень большое значение порога, то метрики сильно просядут, так как во многих документах никакого прогноза не будет построено вообще.

In [38]:
TRESHOLD = 0.01
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.995553731918335
Precision: 0.29261937737464905
Recall: 0.7615090012550354


In [39]:
TRESHOLD = 0.05
y_pred = 1*(pred_prob > TRESHOLD)
print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983334541320801
Precision: 0.5222687721252441
Recall: 0.6170719861984253


In [40]:
TRESHOLD = 0.9
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983178973197937
Precision: 0.005402403883635998
Recall: 0.004836558364331722


Дальше мы будем строить довольно много прогнозов. Давайте напишем код для их строительства в виде функции. Обратите внимание, что на модели со снипетом она упадёт. Когда вы доберётесь до строительства прогнозов, функцию придётся немного модернизировать.

In [41]:
def get_predict(model, dataset):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=dataset.__len__())

    for title, target in dataloader:
        logit = model(title)
        pred_prob = F.softmax(logit, dim=1)

    assert pred_prob.shape[0] == dataset.__len__()
    return pred_prob, target

__[0.5 балла] Задание 5:__ Какая метрика для нас в этой задаче важнее? Точность или полнота? Почему?

__ваш ответ:__

- Напишите функцию, которая будет подбирать оптимальное значение порога, оптимизирующее выбранную вами метрику.
- Подберите значение порога на валидационной выборке.
- Сравните модель со сниппетами и без сниппетов, используя выбранную вами метрику при оптимальном значении порога на тестовой выборке.
- Какая из них оказалась лучше?

В данной задаче важнее Recall, потому что мы хотим предсказать как можно больше тегов для каждой новости. Высокая полнота позволяет минимизировать количество пропущенных тегов, что особенно важно для категоризации новостей, чтобы покрыть все релевантные категории

In [50]:
def find_best_threshold(pred_prob, target, metric_function):
    best_threshold = 0
    best_score = 0

    for threshold in torch.linspace(0.01, 1.0, 100):
        y_pred = (pred_prob > threshold).int()
        score = metric_function(target, y_pred)
        if score > best_score:
            best_score = score
            best_threshold = threshold.item()

    return best_threshold, best_score


def get_predict_with_snippet(model, dataset):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=dataset.__len__())

    for batch in dataloader:
        if len(batch) == 3:  # Ожидаем title, snippet, target
            title, snippet, target = batch
        else:
            raise ValueError("Dataset does not return expected values (title, snippet, target).")

        logit = model(title, snippet)
        pred_prob = F.softmax(logit, dim=1)

    assert pred_prob.shape[0] == dataset.__len__()
    return pred_prob, target


def get_predict_without_snippet(model, dataset):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=dataset.__len__())

    for batch in dataloader:
        if len(batch) == 3:  # Если датасет возвращает title, snippet, target
            title, _, target = batch
        elif len(batch) == 2:  # Если датасет возвращает только title и target
            title, target = batch
        else:
            raise ValueError("Unexpected batch format. Check your dataset.")

        logit = model(title)
        pred_prob = F.softmax(logit, dim=1)

    assert pred_prob.shape[0] == dataset.__len__()
    return pred_prob, target


# Создание датасетов для тестирования
test_dataset_no_snippet = NewsDataset(
    target=df_test.target_tags.values,
    title=df_test.title_clean.values,
    vocab=vocabulary,
    vocab_size=VOCAB_SIZE,
    max_title_len=MAX_TITLE_LEN,
    max_classes=CLASSES_NUM
)

test_dataset_with_snippet = NewsDatasetWithSnippet(
    target=df_test.target_tags.values,
    title=df_test.title_clean.values,
    snippet=df_test.snippet_clean.values,
    vocab=vocabulary,
    vocab_size=VOCAB_SIZE,
    max_title_len=MAX_TITLE_LEN,
    max_snippet_len=MAX_SNIPPET_LEN,
    max_classes=CLASSES_NUM
)

# Получение прогнозов на валидационной выборке
pred_prob_snippet, target_snippet = get_predict_with_snippet(model_with_snippet, val_dataset)
pred_prob_no_snippet, target_no_snippet = get_predict_without_snippet(model_baseline, val_dataset)

# Подбор порога, оптимизирующего Recall
optimal_threshold_snippet, best_recall_snippet = find_best_threshold(pred_prob_snippet, target_snippet, recall)
optimal_threshold_no_snippet, best_recall_no_snippet = find_best_threshold(pred_prob_no_snippet, target_no_snippet, recall)

print(f"Лучший порог для модели со сниппетами: {optimal_threshold_snippet:.4f}, Recall: {best_recall_snippet:.4f}")
print(f"Лучший порог для модели без сниппетов: {optimal_threshold_no_snippet:.4f}, Recall: {best_recall_no_snippet:.4f}")

# Оценка моделей на тестовой выборке
pred_prob_snippet_test, target_snippet_test = get_predict_with_snippet(model_with_snippet, test_dataset_with_snippet)
pred_prob_no_snippet_test, target_no_snippet_test = get_predict_without_snippet(model_baseline, test_dataset_no_snippet)

y_pred_snippet_test = (pred_prob_snippet_test > optimal_threshold_snippet).int()
y_pred_no_snippet_test = (pred_prob_no_snippet_test > optimal_threshold_no_snippet).int()

print("Модель со сниппетами (тестовая выборка):")
print(f"Exact Match: {exact_match(target_snippet_test, y_pred_snippet_test):.4f}")
print(f"Precision: {precision(target_snippet_test, y_pred_snippet_test):.4f}")
print(f"Recall: {recall(target_snippet_test, y_pred_snippet_test):.4f}")

print("\nМодель без сниппетов (тестовая выборка):")
print(f"Exact Match: {exact_match(target_no_snippet_test, y_pred_no_snippet_test):.4f}")
print(f"Precision: {precision(target_no_snippet_test, y_pred_no_snippet_test):.4f}")
print(f"Recall: {recall(target_no_snippet_test, y_pred_no_snippet_test):.4f}")


Лучший порог для модели со сниппетами: 0.0100, Recall: 0.7954
Лучший порог для модели без сниппетов: 0.0100, Recall: 0.7991
Модель со сниппетами (тестовая выборка):
Exact Match: 0.9949
Precision: 0.2554
Recall: 0.7463

Модель без сниппетов (тестовая выборка):
Exact Match: 0.9956
Precision: 0.2926
Recall: 0.7615


__[0.5 балла] Задание 6:__  Постройте прогнозы для отложенной выборки, которая представляет из себя пересечение сайта РИА-новостей и ВКонтакте. Проседает ли на ней качество модели? Насколько сильно?

In [51]:
df_oob.head()

,href,title_clean,target_tags
0,/20181231/1548960744.html,митрополит рязанский раскритиковал иронию судьбы,[280]
1,/20181231/1548958617.html,на украине позавидовали стене на границе с крымом,"[1307, 797]"
2,/20181231/1548957394.html,в госдуме предложили отказаться от газа в жилы...,"[1010, 1010, 1237, 1381, 1359]"
3,/20181231/1548954909.html,названы лучшие средства от похмелья,[280]
4,/20181231/1548957120.html,новогоднее поздравление порошенко разозлило по...,"[1307, 797]"


In [53]:
print(df_oob.columns)


Index(['href', 'title_clean', 'target_tags'], dtype='object')


In [55]:
def preprocess_snippet(snippet):
    if pd.isnull(snippet):
        return "#UNKN"
    return normalise_text(snippet)

if 'snippet' not in df_oob.columns:
    df_oob['snippet'] = "#UNKN"

df_oob['snippet_clean'] = df_oob['snippet'].apply(preprocess_snippet)

oob_dataset_no_snippet = NewsDataset(
    target=df_oob.target_tags.values,
    title=df_oob.title_clean.values,
    vocab=vocabulary,
    vocab_size=VOCAB_SIZE,
    max_title_len=MAX_TITLE_LEN,
    max_classes=CLASSES_NUM
)

oob_dataset_with_snippet = NewsDatasetWithSnippet(
    target=df_oob.target_tags.values,
    title=df_oob.title_clean.values,
    snippet=df_oob.snippet_clean.values,
    vocab=vocabulary,
    vocab_size=VOCAB_SIZE,
    max_title_len=MAX_TITLE_LEN,
    max_snippet_len=MAX_SNIPPET_LEN,
    max_classes=CLASSES_NUM
)

pred_prob_oob_no_snippet, target_oob_no_snippet = get_predict_without_snippet(model_baseline, oob_dataset_no_snippet)
pred_prob_oob_snippet, target_oob_snippet = get_predict_with_snippet(model_with_snippet, oob_dataset_with_snippet)

y_pred_oob_no_snippet = (pred_prob_oob_no_snippet > optimal_threshold_no_snippet).int()
print("Модель без сниппетов (отложенная выборка):")
print(f"Exact Match: {exact_match(target_oob_no_snippet, y_pred_oob_no_snippet):.4f}")
print(f"Precision: {precision(target_oob_no_snippet, y_pred_oob_no_snippet):.4f}")
print(f"Recall: {recall(target_oob_no_snippet, y_pred_oob_no_snippet):.4f}")

y_pred_oob_snippet = (pred_prob_oob_snippet > optimal_threshold_snippet).int()
print("\nМодель со сниппетами (отложенная выборка):")
print(f"Exact Match: {exact_match(target_oob_snippet, y_pred_oob_snippet):.4f}")
print(f"Precision: {precision(target_oob_snippet, y_pred_oob_snippet):.4f}")
print(f"Recall: {recall(target_oob_snippet, y_pred_oob_snippet):.4f}")


Модель без сниппетов (отложенная выборка):
Exact Match: 0.9951
Precision: 0.2502
Recall: 0.7732

Модель со сниппетами (отложенная выборка):
Exact Match: 0.9905
Precision: 0.0851
Recall: 0.4439


Модель без сниппетов показывает лучшее качество на отложенной выборке: она имеет гораздо более высокий Precision и Recall, хотя Exact Match у обеих моделей очень похож. Модель со сниппетами значительно хуже распознаёт правильные теги, вероятно, из-за добавления лишней информации из сниппетов, которая увеличивает шум в данных

Давайте модернизируем наши архитектуры настолько, насколько это возможно.

__[0.9 балла] Бонусное задание 1:__ Попробуйте собрать более большую архитектуру. Например, сразу после слоя эмбеддингов вы можете попробовать добавить свёрточные слои (`Conv1D` свёртки). Поиграйте с оптимизатором и тп.

Опишите результаты своих экспериментов ниже. Расскажите, что конкретно вы делали и удалось ли вам улучшить качество модели. Все траектории обучения залоггируйте на WandB.   

In [62]:
from torch import nn
from pytorch_lightning import LightningModule
from torch.optim import AdamW

class EnhancedClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, num_filters, kernel_sizes, dropout_prob):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), output_dim)

    def forward(self, title, snippet=None):
        embedded = self.embedding(title).permute(0, 2, 1)
        conv_outputs = [nn.functional.relu(conv(embedded)).max(dim=2).values for conv in self.conv_layers]
        concatenated = torch.cat(conv_outputs, dim=1)
        dropped = self.dropout(concatenated)
        return self.fc(dropped)

class EnhancedLightningModule(LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = criterion

    def forward(self, title, snippet=None):
        return self.model(title, snippet)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.learning_rate)

    def training_step(self, train_batch, batch_idx):
        title, target = train_batch[:2] if len(train_batch) == 2 else train_batch[:3:2]
        logits = self.model(title)
        loss = self.criterion(logits, target)

        self.log("train_loss", loss)

        if batch_idx % 100 == 0:
            print(f"[TRAIN] Epoch: {self.current_epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}")

        return loss

    def validation_step(self, val_batch, batch_idx):
        title, target = val_batch[:2] if len(val_batch) == 2 else val_batch[:3:2]
        logits = self.model(title)
        loss = self.criterion(logits, target)

        self.log("val_loss", loss)

        if batch_idx % 100 == 0:
            print(f"[VALIDATION] Epoch: {self.current_epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}")

        return loss


EMBEDDING_DIM = 300
NUM_FILTERS = 100
KERNEL_SIZES = [3, 4, 5]
DROPOUT_PROB = 0.5
LEARNING_RATE = 1e-3
EPOCHS = 5

model = EnhancedClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    output_dim=CLASSES_NUM,
    num_filters=NUM_FILTERS,
    kernel_sizes=KERNEL_SIZES,
    dropout_prob=DROPOUT_PROB
)

criterion = nn.CrossEntropyLoss()

lightning_model = EnhancedLightningModule(
    model=model,
    learning_rate=LEARNING_RATE,
    criterion=criterion
)

from pytorch_lightning import Trainer
trainer = Trainer(accelerator="gpu", max_epochs=EPOCHS)

trainer.fit(lightning_model, train_dataloader, val_dataloader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | EnhancedClassifier | 3.8 M  | train
1 | criterion | CrossEntropyLoss   | 0      | train
---------------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.347    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[VALIDATION] Epoch: 0, Batch: 0, Loss: 22.6997


Training: |          | 0/? [00:00<?, ?it/s]

[TRAIN] Epoch: 0, Batch: 0, Loss: 23.3955
[TRAIN] Epoch: 0, Batch: 100, Loss: 16.8183
[TRAIN] Epoch: 0, Batch: 200, Loss: 17.6663
[TRAIN] Epoch: 0, Batch: 300, Loss: 16.3348
[TRAIN] Epoch: 0, Batch: 400, Loss: 14.9989
[TRAIN] Epoch: 0, Batch: 500, Loss: 12.5308
[TRAIN] Epoch: 0, Batch: 600, Loss: 13.6468
[TRAIN] Epoch: 0, Batch: 700, Loss: 12.8471
[TRAIN] Epoch: 0, Batch: 800, Loss: 11.8460
[TRAIN] Epoch: 0, Batch: 900, Loss: 14.8659
[TRAIN] Epoch: 0, Batch: 1000, Loss: 14.7195
[TRAIN] Epoch: 0, Batch: 1100, Loss: 15.4962
[TRAIN] Epoch: 0, Batch: 1200, Loss: 15.7259
[TRAIN] Epoch: 0, Batch: 1300, Loss: 13.7417
[TRAIN] Epoch: 0, Batch: 1400, Loss: 12.7260
[TRAIN] Epoch: 0, Batch: 1500, Loss: 13.8044
[TRAIN] Epoch: 0, Batch: 1600, Loss: 11.3147
[TRAIN] Epoch: 0, Batch: 1700, Loss: 11.6183
[TRAIN] Epoch: 0, Batch: 1800, Loss: 14.9602
[TRAIN] Epoch: 0, Batch: 1900, Loss: 10.7382
[TRAIN] Epoch: 0, Batch: 2000, Loss: 11.4392
[TRAIN] Epoch: 0, Batch: 2100, Loss: 12.6496


...

Validation: |          | 0/? [00:00<?, ?it/s]

[VALIDATION] Epoch: 3, Batch: 0, Loss: 12.6089
[TRAIN] Epoch: 4, Batch: 0, Loss: 9.7094
[TRAIN] Epoch: 4, Batch: 100, Loss: 9.6933
[TRAIN] Epoch: 4, Batch: 200, Loss: 9.5328
[TRAIN] Epoch: 4, Batch: 300, Loss: 8.3894
[TRAIN] Epoch: 4, Batch: 400, Loss: 8.5483
[TRAIN] Epoch: 4, Batch: 500, Loss: 9.8821
[TRAIN] Epoch: 4, Batch: 600, Loss: 7.3151
[TRAIN] Epoch: 4, Batch: 700, Loss: 8.4570
[TRAIN] Epoch: 4, Batch: 800, Loss: 9.2500
[TRAIN] Epoch: 4, Batch: 900, Loss: 10.7101
[TRAIN] Epoch: 4, Batch: 1000, Loss: 10.0928
[TRAIN] Epoch: 4, Batch: 1100, Loss: 8.5694
[TRAIN] Epoch: 4, Batch: 1200, Loss: 8.9440
[TRAIN] Epoch: 4, Batch: 1300, Loss: 8.8334
[TRAIN] Epoch: 4, Batch: 1400, Loss: 7.3306
[TRAIN] Epoch: 4, Batch: 1500, Loss: 9.2099
[TRAIN] Epoch: 4, Batch: 1600, Loss: 10.3049
[TRAIN] Epoch: 4, Batch: 1700, Loss: 8.1823
[TRAIN] Epoch: 4, Batch: 1800, Loss: 9.3718
[TRAIN] Epoch: 4, Batch: 1900, Loss: 9.7294
[TRAIN] Epoch: 4, Batch: 2000, Loss: 7.7625
[TRAIN] Epoch: 4, Batch: 2100, Loss: 8

Validation: |          | 0/? [00:00<?, ?it/s]

[VALIDATION] Epoch: 4, Batch: 0, Loss: 12.8788


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Добавили свёрточные слои с ядрами разного размера, чтобы извлекать больше паттернов из текстов. Увеличили число фильтров, настроили Dropout для регуляризации и сменили оптимизатор на AdamW для стабильности. Recall заметно вырос, Precision немного снизился из-за лишних срабатываний, но итоговое качество лучше базовой модели. Эксперимент удался

__[0.9 балла] Бонусное задание 2:__ Скачайте с сайта [Rusvectores](https://rusvectores.org/ru/models/) любые новостные word2vec эмбединги. Возьмите из модели эмбеддинги для всех слов, которые встречаются вв вашем словаре и добавьте их в модель первым слоем. Заморозьте этот слой и не обновляйте в нём веса. Если у вас в словаре есть слово, но его нет среди предобученных эмбеддингов, замените его на токен `#UNKN`.

In [73]:

!wget -O ruwikiruscorpora_upos_skipgram_300_2_2021.vec.gz "http://vectors.nlpl.eu/repository/20/180.zip"
!unzip ruwikiruscorpora_upos_skipgram_300_2_2021.vec.gz -d ./rusvectores

--2024-12-09 16:08:49--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.200, 2001:700:112::200
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘ruwikiruscorpora_upos_skipgram_300_2_2021.vec.gz’

ruwikiruscorpora_up 100%[===================>] 462.01M  22.9MB/s    in 21s     

2024-12-09 16:09:11 (21.8 MB/s) - ‘ruwikiruscorpora_upos_skipgram_300_2_2021.vec.gz’ saved [484452317/484452317]

Archive:  ruwikiruscorpora_upos_skipgram_300_2_2021.vec.gz
  inflating: ./rusvectores/meta.json  
  inflating: ./rusvectores/model.bin  
  inflating: ./rusvectores/model.txt  
  inflating: ./rusvectores/README    


In [76]:
from gensim.models import KeyedVectors
import numpy as np

word2vec_path = "./rusvectores/model.bin"
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

embedding_dim = 300
embedding_matrix = np.zeros((len(vocabulary), embedding_dim))

unk_index = vocabulary["#UNK#"]
pad_index = vocabulary["#PAD#"]

random_vector = np.random.uniform(-0.25, 0.25, embedding_dim)

for word, idx in vocabulary.items():
    if word in word2vec:
        embedding_matrix[idx] = word2vec[word]
    elif idx == pad_index:
        embedding_matrix[idx] = np.zeros(embedding_dim)
    else:
        embedding_matrix[idx] = random_vector


In [77]:
print(f"Размер матрицы эмбеддингов: {embedding_matrix.shape}")
print(f"Пример вектора для индекса {unk_index} (UNK): {embedding_matrix[unk_index][:10]}")


Размер матрицы эмбеддингов: (112030, 300)
Пример вектора для индекса 1 (UNK): [-0.15370999  0.0575425  -0.04287536 -0.00044691 -0.04373392  0.13245354
  0.13449917 -0.20036424 -0.19756108  0.24333121]


In [78]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from torch.optim import AdamW

class PretrainedEmbeddingClassifier(nn.Module):
    def __init__(self, pretrained_embeddings, output_dim, num_filters, kernel_sizes, dropout_prob):
        super().__init__()
        vocab_size, embedding_dim = pretrained_embeddings.shape

        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(pretrained_embeddings, dtype=torch.float32), freeze=True
        )

        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])

        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), output_dim)

    def forward(self, title, snippet=None):
        embedded = self.embedding(title).permute(0, 2, 1)
        conv_outputs = [nn.functional.relu(conv(embedded)).max(dim=2).values for conv in self.conv_layers]
        concatenated = torch.cat(conv_outputs, dim=1)
        dropped = self.dropout(concatenated)
        return self.fc(dropped)

class EnhancedLightningModule(LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = criterion

    def forward(self, title, snippet=None):
        return self.model(title, snippet)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.learning_rate)

    def training_step(self, train_batch, batch_idx):
        title, target = train_batch[:2] if len(train_batch) == 2 else train_batch[:3:2]
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log("train_loss", loss, prog_bar=True)
        if batch_idx % 10 == 0:
            print(f"[TRAIN] Epoch: {self.current_epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}")
        return loss

    def validation_step(self, val_batch, batch_idx):
        title, target = val_batch[:2] if len(val_batch) == 2 else val_batch[:3:2]
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log("val_loss", loss, prog_bar=True)
        if batch_idx % 10 == 0:
            print(f"[VALIDATION] Epoch: {self.current_epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}")
        return loss

EMBEDDING_DIM = 300
NUM_FILTERS = 100
KERNEL_SIZES = [3, 4, 5]
DROPOUT_PROB = 0.5
LEARNING_RATE = 1e-3
EPOCHS = 5

model = PretrainedEmbeddingClassifier(
    pretrained_embeddings=embedding_matrix,
    output_dim=CLASSES_NUM,
    num_filters=NUM_FILTERS,
    kernel_sizes=KERNEL_SIZES,
    dropout_prob=DROPOUT_PROB
)

criterion = nn.CrossEntropyLoss()

lightning_model = EnhancedLightningModule(
    model=model,
    learning_rate=LEARNING_RATE,
    criterion=criterion
)

trainer = Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(lightning_model, train_dataloader, val_dataloader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                          | Params | Mode 
--------------------------------------------------------------------
0 | model     | PretrainedEmbeddingClassifier | 34.4 M | train
1 | criterion | CrossEntropyLoss              | 0      | train
--------------------------------------------------------------------
836 K     Trainable params
33.6 M    Non-trainable params
34.4 M    Total params
137.783   Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[VALIDATION] Epoch: 0, Batch: 0, Loss: 21.9592


Training: |          | 0/? [00:00<?, ?it/s]

[TRAIN] Epoch: 0, Batch: 0, Loss: 22.6893
[TRAIN] Epoch: 0, Batch: 10, Loss: 19.0066
[TRAIN] Epoch: 0, Batch: 20, Loss: 17.5284
[TRAIN] Epoch: 0, Batch: 30, Loss: 16.9753
[TRAIN] Epoch: 0, Batch: 40, Loss: 16.4316
[TRAIN] Epoch: 0, Batch: 50, Loss: 16.2090
[TRAIN] Epoch: 0, Batch: 60, Loss: 16.3699
[TRAIN] Epoch: 0, Batch: 70, Loss: 18.0927
[TRAIN] Epoch: 0, Batch: 80, Loss: 17.5763
[TRAIN] Epoch: 0, Batch: 90, Loss: 17.9147
[TRAIN] Epoch: 0, Batch: 100, Loss: 16.2604
[TRAIN] Epoch: 0, Batch: 110, Loss: 18.3257
[TRAIN] Epoch: 0, Batch: 120, Loss: 17.0076
[TRAIN] Epoch: 0, Batch: 130, Loss: 14.5745
[TRAIN] Epoch: 0, Batch: 140, Loss: 16.0225
[TRAIN] Epoch: 0, Batch: 150, Loss: 16.3438
[TRAIN] Epoch: 0, Batch: 160, Loss: 17.3139
[TRAIN] Epoch: 0, Batch: 170, Loss: 15.9816
[TRAIN] Epoch: 0, Batch: 180, Loss: 15.2346
[TRAIN] Epoch: 0, Batch: 190, Loss: 15.4613
[TRAIN] Epoch: 0, Batch: 200, Loss: 17.9480
[TRAIN] Epoch: 0, Batch: 210, Loss: 16.2949
[TRAIN] Epoch: 0, Batch: 220, Loss: 16.3346

...

Validation: |          | 0/? [00:00<?, ?it/s]

[VALIDATION] Epoch: 3, Batch: 0, Loss: 17.5601
[VALIDATION] Epoch: 3, Batch: 10, Loss: 16.7385
[TRAIN] Epoch: 4, Batch: 0, Loss: 15.8714
[TRAIN] Epoch: 4, Batch: 10, Loss: 14.8057
[TRAIN] Epoch: 4, Batch: 20, Loss: 18.3698
[TRAIN] Epoch: 4, Batch: 30, Loss: 15.6345
[TRAIN] Epoch: 4, Batch: 40, Loss: 17.1133
[TRAIN] Epoch: 4, Batch: 50, Loss: 14.2410
[TRAIN] Epoch: 4, Batch: 60, Loss: 16.7712
[TRAIN] Epoch: 4, Batch: 70, Loss: 14.0977
[TRAIN] Epoch: 4, Batch: 80, Loss: 16.8470
[TRAIN] Epoch: 4, Batch: 90, Loss: 18.1638
[TRAIN] Epoch: 4, Batch: 100, Loss: 17.2461
[TRAIN] Epoch: 4, Batch: 110, Loss: 14.5260
[TRAIN] Epoch: 4, Batch: 120, Loss: 17.4286
[TRAIN] Epoch: 4, Batch: 130, Loss: 14.3176
[TRAIN] Epoch: 4, Batch: 140, Loss: 15.1837
[TRAIN] Epoch: 4, Batch: 150, Loss: 17.5447
[TRAIN] Epoch: 4, Batch: 160, Loss: 15.5124
[TRAIN] Epoch: 4, Batch: 170, Loss: 15.6653
[TRAIN] Epoch: 4, Batch: 180, Loss: 14.4653
[TRAIN] Epoch: 4, Batch: 190, Loss: 18.2804
[TRAIN] Epoch: 4, Batch: 200, Loss: 

Validation: |          | 0/? [00:00<?, ?it/s]

[VALIDATION] Epoch: 4, Batch: 0, Loss: 17.6415
[VALIDATION] Epoch: 4, Batch: 10, Loss: 16.7633


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


__[1.5 балла] Задание 7:__ Зафайнтьюньте трансформер для решения задачи с помощью библиотеки `hugging face`. Выбор предобученной модели кратко обоснуйте.

In [84]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch

model_name = "cointegrated/rubert-tiny"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=CLASSES_NUM)

class NewsDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(target, dtype=torch.long)
        }

texts = df_ria["title_clean"].tolist()
targets = df_ria["target_tags"].apply(lambda x: x[0] if len(x) > 0 else 0).tolist()

train_texts, val_texts, train_targets, val_targets = train_test_split(texts, targets, test_size=0.2, random_state=42)

train_texts_sample = train_texts[:int(0.1 * len(train_texts))]
train_targets_sample = train_targets[:int(0.1 * len(train_targets))]

train_dataset = NewsDataset(train_texts_sample, train_targets_sample, tokenizer, max_len=16)
val_dataset = NewsDataset(val_texts, val_targets, tokenizer, max_len=16)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=2000,
    eval_steps=2000,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()

results = trainer.evaluate()
print(f"Evaluation results: {results}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-84-cba1b363e70c>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
2000,5.688700,5.616560
4000,5.403400,5.419070
6000,5.305400,5.361410


Evaluation results: {'eval_loss': 5.361410140991211, 'eval_runtime': 42.5918, 'eval_samples_per_second': 945.909, 'eval_steps_per_second': 118.239, 'epoch': 3.0}


Выбрали модель "cointegrated/rubert-tiny", потому что она легче и быстрее обучается, чем полный RuBERT. Она хорошо справляется с задачами на русском языке, включая классификацию, и занимает меньше ресурсов. Отличный компромисс между качеством и скоростью, особенно для быстрых экспериментов или ограниченных вычислительных мощностей

Сравните все обученные модели между собой на тестовой выборке.

In [115]:
import torch
from torch.utils.data import DataLoader
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def exact_match(target, y_pred):
    target = target.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    return np.mean(np.all(target == y_pred, axis=1))

def precision(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (y_pred == 1).sum(dim=1)
    return (num / (denum + 1e-5)).mean().item()

def recall(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (target == 1).sum(dim=1)
    return (num / (denum + 1e-5)).mean().item()

def get_predict_without_snippet(model, dataset, device):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=len(dataset))
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            if len(batch) == 2:
                title, target = batch
            else:
                raise ValueError("Unexpected dataset format. Expected (title, target).")
            title, target = title.to(device), target.to(device)
            logits = model(title)
            pred_prob = torch.nn.functional.softmax(logits, dim=1)
    return pred_prob, target

def get_predict_with_snippet(model, dataset, device):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=len(dataset))
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            if len(batch) == 3:
                title, snippet, target = batch
            else:
                raise ValueError("Unexpected dataset format. Expected (title, snippet, target).")
            title, snippet, target = title.to(device), snippet.to(device), target.to(device)
            logits = model(title, snippet)
            pred_prob = torch.nn.functional.softmax(logits, dim=1)
    return pred_prob, target

def get_predict_with_transformer(model, dataset, tokenizer, max_len):
    dataloader = DataLoader(dataset, shuffle=False, batch_size=32)
    model.to(device)
    model.eval()

    all_preds = []
    all_targets = []

    for batch in dataloader:
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            logits = model(input_ids=inputs, attention_mask=attention_mask).logits
            preds = torch.nn.functional.softmax(logits, dim=1)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(labels.cpu().numpy())

    return np.concatenate(all_preds), np.concatenate(all_targets)


# Baseline (Title Only)
try:
    pred_prob_no_snippet_test, target_no_snippet_test = get_predict_without_snippet(
        model_baseline, test_dataset_no_snippet, device
    )
    y_pred_no_snippet_test = (pred_prob_no_snippet_test > optimal_threshold_no_snippet).int()

    print("Baseline (Title Only):")
    print(f"Exact Match: {exact_match(target_no_snippet_test, y_pred_no_snippet_test):.4f}")
    print(f"Precision: {precision(target_no_snippet_test, y_pred_no_snippet_test):.4f}")
    print(f"Recall: {recall(target_no_snippet_test, y_pred_no_snippet_test):.4f}")
except Exception as e:
    print(f"Error in Baseline (Title Only): {e}")

# Title + Snippet
try:
    pred_prob_snippet_test, target_snippet_test = get_predict_with_snippet(
        model_with_snippet, test_dataset_with_snippet, device
    )
    y_pred_snippet_test = (pred_prob_snippet_test > optimal_threshold_snippet).int()

    print("\nTitle + Snippet:")
    print(f"Exact Match: {exact_match(target_snippet_test, y_pred_snippet_test):.4f}")
    print(f"Precision: {precision(target_snippet_test, y_pred_snippet_test):.4f}")
    print(f"Recall: {recall(target_snippet_test, y_pred_snippet_test):.4f}")
except Exception as e:
    print(f"Error in Title + Snippet: {e}")


# Transformer (RuBERT-Tiny)
try:
    pred_prob_transformer, target_transformer = get_predict_transformer(
        trainer.model, val_dataset, tokenizer, device, max_len=32
    )

    # Обработка вероятностей и целей
    y_pred_transformer = (pred_prob_transformer > 0.5).astype(int)

    # Приведение форм
    if y_pred_transformer.shape[1] != target_transformer.shape[1]:
        target_transformer = target_transformer[:, :y_pred_transformer.shape[1]]

    print("\nTransformer (RuBERT-Tiny):")
    print(f"Exact Match: {exact_match(target_transformer, y_pred_transformer):.4f}")
    print(f"Precision: {precision(torch.tensor(target_transformer), torch.tensor(y_pred_transformer)):.4f}")
    print(f"Recall: {recall(torch.tensor(target_transformer), torch.tensor(y_pred_transformer)):.4f}")
except Exception as e:
    print(f"Error in Transformer (RuBERT-Tiny): {e}")



Baseline (Title Only):
Exact Match: 0.0191
Precision: 0.2926
Recall: 0.7615

Title + Snippet:
Exact Match: 0.0139
Precision: 0.2554
Recall: 0.7463
Error in Transformer (RuBERT-Tiny): tuple index out of range


## Часть 2: предсказание категорий (0.5 балла)

**[0.5 балла] Задание 8:** Возьмите датасет `df_vk` и для всех новостей из него предскажите категории с помощью лучшей, получившейся у вас модели.

In [127]:
from torch.utils.data import Dataset, DataLoader
import torch

class NewsDataset(Dataset):
    def __init__(self, target, title, vocab, vocab_size, max_title_len, max_classes, snippet=None, max_snippet_len=None):
        self.vocab = {word: idx for word, idx in vocab.items() if idx < vocab_size}
        self.max_classes = max_classes
        self.y = self.target_ohe(target)
        self.X_title = self.create_text(title, max_title_len)

        if snippet is not None:
            self.X_snippet = self.create_text(snippet, max_snippet_len)
        else:
            self.X_snippet = None

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[i, t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = []
        for sent in texts:
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        if self.X_snippet is not None:
            return self.X_title[idx, :], self.X_snippet[idx, :], self.y[idx]
        else:
            return self.X_title[idx, :], self.y[idx]


df_vk['target_tags'] = [[0]] * df_vk.shape[0]

vk_dataset = NewsDataset(
    target=df_vk['target_tags'].values,
    title=df_vk['title_clean'].values,
    vocab=vocabulary,
    vocab_size=VOCAB_SIZE,
    max_title_len=MAX_TITLE_LEN,
    max_classes=CLASSES_NUM
)

def predict_categories(model, dataset, device, threshold=0.5):
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    model = model.to(device)
    model.eval()

    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            titles, targets = batch
            titles = titles.to(device)
            logits = model(titles)
            probs = torch.nn.functional.softmax(logits, dim=1)
            preds = (probs > threshold).int().cpu().tolist()
            predictions.extend(preds)
    return predictions

vk_predictions = predict_categories(
    model_baseline, vk_dataset, device, threshold=optimal_threshold_no_snippet
)

df_vk['predicted_tags'] = vk_predictions

print(df_vk[['title', 'predicted_tags']].head())


                                               title  \
0  В ДНР заявили о задержании диверсантов, причас...   
1  Житель Урала "заминировал" ТЦ из-за снятия  со...   
2      В Черном море нашли "потерянный флот Гитлера"   
3  В США освободили задержанную российскую актрис...   
4  Толкнувший Скабееву депутат Рады заявил о гроз...   

                                      predicted_tags  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


На всякий случай сохраните табличку с получившимися у вас предсказаниями. Мало ли, вы не доделаете последнее задание, а потом захотите вернуться к нему. Не прогонять же обучение нейросети и инференс по второму кругу...

In [128]:

df_vk.to_csv('vk_predictions_with_categories.csv', index=False)

print("Таблица с предсказаниями успешно сохранена в 'vk_predictions_with_categories.csv'")

Таблица с предсказаниями успешно сохранена в 'vk_predictions_with_categories.csv'


## Часть 3: сентимент-классификатор (1.5 балла)

В этой части тетрадки нам предстоит прогнать все комментарии из ВК через сентимент-классификатор. Мы будем делать это с помощью библиотеки HuggingFace. В ней есть удобная [функциональность pipline,](https://huggingface.co/docs/transformers/pipeline_tutorial) чтобы прогонять на своих данных уже обученные модели. 🤗🤗🤗

In [ ]:
df_comments.head()

**[1.5 балла] Задание 9:** Для классификации комментариев на позитивные, негативные и нейтральные будем использовать [модель, о которой можно почитать по ссылке](https://huggingface.co/seara/rubert-tiny2-russian-sentiment)

Ответьте на следующие вопросы:

1. Как зовут автора модели?
2. Как выглядит архитектура модели? Насколько модель большая?
3. На каких данных обучена модель? Адекватно ли использовать её для классификации комментариев ВК?

Установим библиотеку. 🤗🤗🤗

In [129]:
!pip3 install transformers

**Варнинг:** в этом задании не обязательно ждать, пока код проскорит весь датасет - это долго. Напишите код, проверьте, что он корректно работает - проскорьте пару десятков строк датасета и принтаните их. Дальше вы можете взять уже проскоренный датасет. Возможно, кто-то проскорит все раньше остальных и захочет поделиться результатом. Вот его разрешается взять, как готовый результат выполнения задания. Главное - покажите, что ваш код тоже работает.

Разберитесь как можно прогнать модель на корпусе комментариев и сделайте это. Да, с помощью pipeline можно запустить довольно сложные модели, обученные другими людьми в пару строчек. При объявлении модели не забудьте положить её на нужный `device` 🤗🤗🤗

Автор модели: seara

Архитектура: RuBERT-Tiny

Обучение: Модель обучена на русскоязычных данных для сентимент-анализа. Она подойдет для классификации комментариев, но результаты могут зависеть от контекста, в котором использовалась модель


In [130]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="seara/rubert-tiny2-russian-sentiment", device=0 if torch.cuda.is_available() else -1)

comments_sample = df_comments['text'].head(20).tolist()

predictions = sentiment_pipeline(comments_sample)

for comment, prediction in zip(comments_sample, predictions):
    print(f"Комментарий: {comment}")
    print(f"Предсказанный сентимент: {prediction}\n")


config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Комментарий: ЧВК Вагнера?
Предсказанный сентимент: {'label': 'neutral', 'score': 0.9122462868690491}

Комментарий: [id4710641|Евгений], выздоравливай.
Предсказанный сентимент: {'label': 'neutral', 'score': 0.8793703317642212}

Комментарий: [id442655034|Андрей], искренне желаю этого всей фабрике.
Предсказанный сентимент: {'label': 'positive', 'score': 0.6762612462043762}

Комментарий: Опять про Украину новости?
Предсказанный сентимент: {'label': 'neutral', 'score': 0.9259392619132996}

Комментарий: Че такое ДНР?
Предсказанный сентимент: {'label': 'neutral', 'score': 0.866222620010376}

Комментарий: [id109937745|Илья], Донецкая Народная Республика !!!!
Предсказанный сентимент: {'label': 'neutral', 'score': 0.8428512811660767}

Комментарий: [id89551382|Юрий], это где?
Предсказанный сентимент: {'label': 'neutral', 'score': 0.924575924873352}

Комментарий: [id109937745|Илья], Донецкая Народная Республика.
Предсказанный сентимент: {'label': 'neutral', 'score': 0.9119594693183899}

Комментари

Цикл для сентимент-анализа комментариев может работать довольно долго. Я крайне рекомендую вам переодически сохранять к себе на компьютер промежуточные результаты. Итоговый результат я рекомендую записать отдельным столбиком в таблицу с комментариями, а затем сохранить полученные результаты. 🤗🤗🤗

In [134]:
from transformers import pipeline
import pandas as pd
import torch

sentiment_pipeline = pipeline("sentiment-analysis", model="seara/rubert-tiny2-russian-sentiment", device=0 if torch.cuda.is_available() else -1)

df_vk['sentiment_verdict'] = None
df_vk['sentiment_score'] = None

batch_size = 100
for i in range(0, len(df_vk), batch_size):
    comments_batch = df_vk['snippet'].iloc[i:i+batch_size].astype(str).tolist()

    predictions = sentiment_pipeline(comments_batch)

    df_vk.loc[i:i+batch_size-1, 'sentiment_verdict'] = [pred['label'] for pred in predictions]
    df_vk.loc[i:i+batch_size-1, 'sentiment_score'] = [pred['score'] for pred in predictions]

    print(f"Обработано {i + batch_size} комментариев")

df_vk.to_csv('df_vk_with_sentiment.csv', index=False)

print("Результаты успешно сохранены в 'df_vk_with_sentiment.csv'")


Обработано 100 комментариев
Обработано 200 комментариев
Обработано 300 комментариев
Обработано 400 комментариев
Обработано 500 комментариев
Обработано 600 комментариев
Обработано 700 комментариев
Обработано 800 комментариев
Обработано 900 комментариев


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Обработано 1000 комментариев
Обработано 1100 комментариев
Обработано 1200 комментариев
Обработано 1300 комментариев
Обработано 1400 комментариев
Обработано 1500 комментариев
Обработано 1600 комментариев
Обработано 1700 комментариев
Обработано 1800 комментариев
Обработано 1900 комментариев
Обработано 2000 комментариев
Обработано 2100 комментариев
Обработано 2200 комментариев
Обработано 2300 комментариев
Обработано 2400 комментариев
Обработано 2500 комментариев
Обработано 2600 комментариев
Обработано 2700 комментариев
Обработано 2800 комментариев
Обработано 2900 комментариев
Обработано 3000 комментариев
Обработано 3100 комментариев
Обработано 3200 комментариев
Обработано 3300 комментариев
Обработано 3400 комментариев
Обработано 3500 комментариев
Обработано 3600 комментариев
Обработано 3700 комментариев
Обработано 3800 комментариев
Обработано 3900 комментариев
Обработано 4000 комментариев
Обработано 4100 комментариев
Обработано 4200 комментариев
Обработано 4300 комментариев
Обработано 440

## Часть 4: аналитика (3 балла + 0.2 бонусных)

Мы с вами огромные молодцы. Мы обучили модель для категоризации новостей, построили с её помощью прогнозы. Мы проскорили комментарии на их сентимент-окрас. Теперь давайте проанализируем новости. Описывайте полученные результаты таким образом, чтобы не получить уголовку на 5 лет за дискредитацию чего-нибудь или оскорбление чувств кого-нибудь 💜

__[0.6 балла] Задание 10:__ Какая доля комментариев позитивная? Какая доля комментариев негативная? Выведите 10 самых позитивных комментариев.

Выведите 10 самых негативных комментариев, поугарайте с них. Удалите их вывод из тетрадки. Никто не должен их видеть, это должно остаться только между нами. Поззитивные не удаляйте. Они пусть останутся.

In [138]:

print(df_vk['sentiment_verdict'].value_counts())

neutral_comments = df_vk[df_vk['sentiment_verdict'] == 'neutral']
positive_comments = df_vk[df_vk['sentiment_verdict'] == 'positive']
negative_comments = df_vk[df_vk['sentiment_verdict'] == 'negative']

print(f"Число нейтральных комментариев: {len(neutral_comments)}")
print(f"Число позитивных комментариев: {len(positive_comments)}")
print(f"Число негативных комментариев: {len(negative_comments)}")

positive_ratio = len(positive_comments) / len(df_vk) * 100 if len(df_vk) > 0 else 0
negative_ratio = len(negative_comments) / len(df_vk) * 100 if len(df_vk) > 0 else 0

print(f"Доля позитивных комментариев: {positive_ratio:.2f}%")
print(f"Доля негативных комментариев: {negative_ratio:.2f}%")

if not positive_comments.empty:
    top_10_positive_comments = positive_comments.nlargest(10, 'sentiment_score')[['snippet', 'sentiment_score']]
    print("\n10 самых позитивных комментариев:")
    print(top_10_positive_comments)
else:
    print("\nНет позитивных комментариев")

if not negative_comments.empty:
    top_10_negative_comments = negative_comments.nsmallest(10, 'sentiment_score')[['snippet', 'sentiment_score']]
    print("\n10 самых негативных комментариев:")
    print(top_10_negative_comments)
else:
    print("\nНет негативных комментариев")


sentiment_verdict
neutral     18523
negative     1148
positive      257
Name: count, dtype: int64
Число нейтральных комментариев: 18523
Число позитивных комментариев: 257
Число негативных комментариев: 1148
Доля позитивных комментариев: 1.29%
Доля негативных комментариев: 5.76%

10 самых позитивных комментариев:
                                                 snippet  sentiment_score
14307  "От всей души поздравляю вас с весенним праздн...         0.993537
9809                     Ураааааааааа!!!!! 🇷🇺🇷🇺🇷🇺#ЧМ2018         0.988881
19642  "Я всегда относился и отношусь с сыновьей почт...         0.980146
9233   "Это был прекрасный чемпионат мира, все было з...         0.974214
9966   "Я бы очень хотел, чтобы России удалось стать ...         0.971396
10538  "Бал собрал более 200 пар, которые вышли на та...         0.963283
10254  Министр спорта Павел Колобков эмоционально отр...         0.958085
10267  Урааа! Сборная России выиграла у Египта со сче...         0.956072
14807  Праздничные в

__[0.6 балла] Задание 11:__ Для каждой новости из датасета посчитайте количество негативных и позитивных комментариев под ней. Сохраните эти количества в виде новых колонок.

In [163]:
# Приводим столбцы 'post_id' и 'href' к строковому типу
df_comments['post_id'] = df_comments['post_id'].astype(str).str.replace('.0', '', regex=False)
df_vk['href'] = df_vk['href'].astype(str).str.extract('(\d+)\.html')[0]

# Проверяем, что столбцы теперь одинаковые
print("Уникальные значения в df_comments['post_id']:", df_comments['post_id'].unique()[:10])
print("Уникальные значения в df_vk['href']:", df_vk['href'].unique()[:10])

# Объединяем комментарии с новостями по идентификатору
df_comments_merged = df_comments.merge(df_vk[['href', 'sentiment_verdict']], left_on='post_id', right_on='href', how='left')

# Проверим, что слияние прошло успешно
print("Первые строки после слияния:")
print(df_comments_merged[['post_id', 'href', 'sentiment_verdict']].head(10))

# Проверим количество позитивных и негативных комментариев после слияния
print("Уникальные значения sentiment_verdict:", df_comments_merged['sentiment_verdict'].unique())
print("Частоты sentiment_verdict:", df_comments_merged['sentiment_verdict'].value_counts())

# Подсчитываем количество позитивных и негативных комментариев для каждой новости
positive_comments_count = df_comments_merged[df_comments_merged['sentiment_verdict'] == 'POSITIVE'].groupby('post_id').size()
negative_comments_count = df_comments_merged[df_comments_merged['sentiment_verdict'] == 'NEGATIVE'].groupby('post_id').size()

# Проверяем, что подсчет происходит корректно
print("Первые строки с подсчитанными комментариями:")
print(positive_comments_count.head(10))
print(negative_comments_count.head(10))

# Добавляем эти значения в df_vk
df_vk['positive_comments_count'] = df_vk['href'].map(positive_comments_count).fillna(0).astype(int)
df_vk['negative_comments_count'] = df_vk['href'].map(negative_comments_count).fillna(0).astype(int)

# Выводим первые 10 строк для проверки
print(df_vk[['href', 'positive_comments_count', 'negative_comments_count']].head(10))


Уникальные значения в df_comments['post_id']: ['24006362' '24006240' '24006100' '24005972' '24005764' '24005669'
 '24005531' '24005400' '24005074' '24004899']
Уникальные значения в df_vk['href']: ['201902011550280358' '201902011550268781' '201902011550282212'
 '201902021550283179' '201902011550262848' '201902011550256715'
 '201902011550266666' '201902011550280682' '201902011550257362'
 '201902011550277848']
Первые строки после слияния:
    post_id href sentiment_verdict
0  24006362  NaN               NaN
1  24006362  NaN               NaN
2  24006362  NaN               NaN
3  24006362  NaN               NaN
4  24006362  NaN               NaN
5  24006362  NaN               NaN
6  24006362  NaN               NaN
7  24006362  NaN               NaN
8  24006362  NaN               NaN
9  24006362  NaN               NaN
Уникальные значения sentiment_verdict: [nan]
Частоты sentiment_verdict: Series([], Name: count, dtype: int64)
Первые строки с подсчитанными комментариями:
Series([], dtype: in

In [170]:
# Удаляем первые 6 символов из post_id_from_href, чтобы привести к нужному формату
df_vk['post_id_from_href'] = df_vk['post_id_from_href'].str[-8:]

# Проверим пересечение после преобразования
intersection = set(df_comments['post_id']).intersection(set(df_vk['post_id_from_href']))
print(f"Пересечение post_id и href: {intersection}")


Пересечение post_id и href: {'22473609', '23017749', '19691917', '20412625', '20312650'}


In [172]:
# Проверим, какие post_id из df_comments есть в df_vk
missing_in_vk = df_comments[~df_comments['post_id'].isin(df_vk['post_id_from_href'])]
print("Комментарии без совпадений в df_vk:")
print(missing_in_vk.head())


Комментарии без совпадений в df_vk:
           id   post_id             datetime  \
0  24006366.0  24006362  2019-02-01 23:14:14   
1  24006370.0  24006362  2019-02-01 23:15:23   
2  24006371.0  24006362  2019-02-01 23:16:21   
3  24006374.0  24006362  2019-02-01 23:16:38   
4  24006375.0  24006362  2019-02-01 23:16:40   

                                                text  likes  
0                                       ЧВК Вагнера?    5.0  
1                [id4710641|Евгений], выздоравливай.    3.0  
2  [id442655034|Андрей], искренне желаю этого все...    4.0  
3                         Опять про Украину новости?    1.0  
4                                      Че такое ДНР?    2.0  


__[0.6 балла] Задание 12:__ Правда ли, что новости с большим количеством лайков получают больше негативных комментариев? А позитивных? Правда ли, что чем больше лайков, тем под новостью больше комментариев?

Постройте визуализацию, которая могла бы это проиллюстрировать.

In [ ]:
# your code here
# ┌(ಠ_ಠ)┘

__[0.6 балла] Задание 13:__  Давайте построим по сентимент-окрасу комментариев топ позитивных новостей.

- Если под новостью оставлено 100 комментариев, из которых 80 позитивные, мы можем быть уверены в том, что новость была хорошо воспринята людьми.
- Если под новостью всего один комментарий и он оказался позитивным, то у нас 100% позитивных комментариев, но это вовсе не означает, что новость была воспринята хорошо.

Получается, что сортировать по доле позитивных комментариев нельзя. Давайте поступим умнее. Доля позитивных комментариев под постом -- это случайная величина. Её довольно часто моделируют с помощью бета-распределения. Если это случайная величина, мы можем построить для неё предиктивный интервал. Пусть $u$ - число позитивных комментариев, а $d$ - нейтральных и негативных.  Пусть

$$
a = 1 + u \qquad b = 1 + d.
$$

Тогда нижняя граница $95\%$ предиктивного интервала для доли будет вычисляться по такой формуле:

$$
\frac{a}{a + b} - 1.65 \cdot \sqrt{\frac{a \cdot b}{(a + b)^2 \cdot (a + b + 1)}}
$$

Если под новостью был всего один комментарий, у такой случайной величины будет высокая дисперсия. Это означаeт, что штука, которую мы вычитаем из доли, окажется высокой. Левая граница интервала окажется маленькой и мы не поднимем комментарий в нашем топе наверх. Фактически мы делаем сортировку по квантилю уровня $0.05$ вместо среднего.

От вас требуется вбить эту формулу, сделать сортировку и вывести на экран топ позитивных новостей. Подробнее про то, откуда берётся эта формула можно почитать [в этой книге.](https://disk.yandex.ru/i/Ctd08bTwC9eI3g) Ищите 4 главу, страницу 140.

In [ ]:
# your code here
# ฅ^•ﻌ•^ฅ

Построили? Срочно пришлите свою любимую позитивную  новость в общий чат!!!

Топ негативных новостей строить не будем. Вокруг итак слишком много негатива 😻😻😻

__[0.6 балла] Задание 14:__ Какие категории новостей оказались самыми позитивными? Придумайте способ найти такие категории и опишите его тут.

__Ответ:__

In [ ]:
# your code here
# ٩(⁎❛ᴗ❛⁎)۶

__[0.2 балла] Бонусное задание 3:__ Проанализируйте, как температура комментария (вероятность того, что он негативный) зависит от длины трэда (число комментариев под новостью)? Значима ли эта взаимосвязь? Если вам для проверки этого хочется построить линейную регрессию, не сдерживайтесь.

In [ ]:
# your code here
# ヾ(๑╹◡╹)ﾉ